# Предсказание цены автомобиля

Сервис по продаже автомобилей с пробегом разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В нашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Нам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

# 1. Подготовка данных

### Описание данных

#### Признаки
- *DateCrawled* — дата скачивания анкеты из базы
- *VehicleType* — тип автомобильного кузова
- *RegistrationYear* — год регистрации автомобиля
- *Gearbox* — тип коробки передач
- *Power* — мощность (л. с.)
- *Model* — модель автомобиля
- *Kilometer* — пробег (км)
- *RegistrationMonth* — месяц регистрации автомобиля
- *FuelType* — тип топлива
- *Brand* — марка автомобиля
- *NotRepaired* — была машина в ремонте или нет
- *DateCreated* — дата создания анкеты
- *NumberOfPictures* — количество фотографий автомобиля
- *PostalCode* — почтовый индекс владельца анкеты (пользователя)
- *LastSeen* — дата последней активности пользователя


#### Целевой признак
 *Price* — цена (евро)

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from catboost import CatBoostRegressor, Pool, cv
import lightgbm as lgb

cars = pd.read_csv('/datasets/autos.csv')
cars

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354364,2016-03-21 09:50:58,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes,2016-03-21 00:00:00,0,2694,2016-03-21 10:42:49
354365,2016-03-14 17:48:27,2200,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,2016-03-14 00:00:00,0,39576,2016-04-06 00:46:52
354366,2016-03-05 19:56:21,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no,2016-03-05 00:00:00,0,26135,2016-03-11 18:17:12
354367,2016-03-19 18:57:12,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no,2016-03-19 00:00:00,0,87439,2016-04-07 07:15:26


In [2]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


В данных наблюдается достаточно много пропусков, в столбцах *Price* и *Power* замечены нулевые значения, что тоже является по своей сути пропусками, т.к. у авто не может быть нулевой мощности или стоимости.

In [3]:
cars.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


В столбце *RegistrationYear* замечены аномальные значения: слишком маленький год, например, 1000 (вряд ли 1000-ом году существовали автомобили, также аномально высокие значения - 9999 год. В столбце *Power* также присутствуют аномально высокие значения: 20000 л.с. - таких значений не бывает даже у танков, хотя может это машина из будущего...

Уберем все машины из будущего и те, что раньше 1900 годя, т.к. сложно говорить о серийном выпуске автомобилей в то время, тогда он только зарождался.

In [4]:
cars_new = cars.loc[(cars['RegistrationYear'] > 1945) & (cars['RegistrationYear'] < 2020)]

In [5]:
# Уберем строки с нулевой ценой
cars_new = cars_new.loc[cars_new['Price'] > 0]

In [6]:
# Уберем нулевую мощность и аномально высокую. Суперкары имеют мощность в районе 1600 л.с. и до 2100 л.с. (но это уже единичные модели)
# Ограничемся значением 1000 л.с., что, однако, тоже может быть много для наших данных.
cars_new = cars_new.loc[(cars_new['Power'] < 1000) & (cars_new['Power'] > 0)]

Заменим пропуски в категориальных столбцах значениями "unk".

In [8]:
cat_columns = ["VehicleType", "Gearbox", "Model", "FuelType", "Brand", "NotRepaired"]

In [9]:
cars_new[cat_columns] = cars_new[cat_columns].fillna('unk')

In [10]:
cars_new

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,unk,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,unk,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
5,2016-04-04 17:36:23,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,2016-04-04 00:00:00,0,33775,2016-04-06 19:17:07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354361,2016-03-09 13:37:43,5250,unk,2016,auto,150,159,150000,12,unk,alfa_romeo,no,2016-03-09 00:00:00,0,51371,2016-03-13 01:44:13
354362,2016-03-19 19:53:49,3200,sedan,2004,manual,225,leon,150000,5,petrol,seat,yes,2016-03-19 00:00:00,0,96465,2016-03-19 20:44:43
354366,2016-03-05 19:56:21,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no,2016-03-05 00:00:00,0,26135,2016-03-11 18:17:12
354367,2016-03-19 18:57:12,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no,2016-03-19 00:00:00,0,87439,2016-04-07 07:15:26


In [11]:
cars_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306872 entries, 1 to 354368
Data columns (total 16 columns):
DateCrawled          306872 non-null object
Price                306872 non-null int64
VehicleType          306872 non-null object
RegistrationYear     306872 non-null int64
Gearbox              306872 non-null object
Power                306872 non-null int64
Model                306872 non-null object
Kilometer            306872 non-null int64
RegistrationMonth    306872 non-null int64
FuelType             306872 non-null object
Brand                306872 non-null object
NotRepaired          306872 non-null object
DateCreated          306872 non-null object
NumberOfPictures     306872 non-null int64
PostalCode           306872 non-null int64
LastSeen             306872 non-null object
dtypes: int64(7), object(9)
memory usage: 39.8+ MB


In [12]:
cars_new.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,306872.000000,306872.000000,306872.000000,306872.000000,306872.000000,306872.0,306872.000000
mean,4810.601691,2003.310064,120.354451,128430.062045,5.964093,0.0,51154.805779
std,4586.557841,6.900593,54.648570,36725.905935,3.599911,0.0,25768.489407
min,1.000000,1947.000000,1.000000,5000.000000,0.000000,0.0,1067.000000
25%,1300.000000,1999.000000,75.000000,125000.000000,3.000000,0.0,30916.000000
50%,3100.000000,2003.000000,110.000000,150000.000000,6.000000,0.0,50170.000000
75%,6950.000000,2008.000000,150.000000,150000.000000,9.000000,0.0,71717.000000
max,20000.000000,2019.000000,999.000000,150000.000000,12.000000,0.0,99998.000000


In [13]:
#Уберем Price меньше 100 евро и Power меньше 10 л.с. (нашел на авто.ру минимальную мощность 10 л.с.)
cars_new = cars_new.loc[(cars_new['Price'] > 100) & (cars_new['Power'] >= 10)]

Столбцы *DateCrawled*, *RegistrationMonth*, *DateCreated*, *NumberOfPictures*, *PostalCode*, *LastSeen* будут лишними для обучения модели, так как не являются характеристиками автомобилей.

In [14]:
features_train, features_test, target_train, target_test = train_test_split(
    cars_new.drop(['DateCrawled', 'Price', 'RegistrationMonth', 'DateCreated', 'NumberOfPictures', 'PostalCode', 'LastSeen'], axis=1), cars_new.Price, test_size=0.25, random_state=12345)

Данные подготовлены для обучения моделей.

# 2. Обучение моделей

In [15]:
#Обучим модель градиентного бустинга из catboost применив кросс-валидацию
cv_cars = Pool(data=features_train,
              label=target_train,
              cat_features=cat_columns)

In [16]:
#Сохраняем данные специальной функцией Pool для избежания лишнего копирования
pool_test = Pool(data=features_test, label=target_test, cat_features=cat_columns)

In [17]:
#Параметры модели
params = {'iterations': 1000,
         'depth': 6,
         'loss_function': 'RMSE',
          'learning_rate': 0.1,
         'verbose': 20,
         'random_state': 12345}

In [18]:
scores = cv(cv_cars, params, fold_count=3, plot=False, as_pandas=True)

0:	learn: 6121.9946045	test: 6121.7997619	best: 6121.7997619 (0)	total: 2.92s	remaining: 48m 35s
20:	learn: 2248.5585344	test: 2251.2046728	best: 2251.2046728 (20)	total: 45.1s	remaining: 35m 4s
40:	learn: 1920.9235629	test: 1923.5145233	best: 1923.5145233 (40)	total: 1m 21s	remaining: 31m 48s
60:	learn: 1846.4238682	test: 1851.0629407	best: 1851.0629407 (60)	total: 2m 1s	remaining: 31m 10s
80:	learn: 1803.1265377	test: 1810.8279195	best: 1810.8279195 (80)	total: 2m 40s	remaining: 30m 17s
100:	learn: 1774.1477301	test: 1784.2715969	best: 1784.2715969 (100)	total: 3m 19s	remaining: 29m 39s
120:	learn: 1751.0511377	test: 1763.5263829	best: 1763.5263829 (120)	total: 3m 58s	remaining: 28m 55s
140:	learn: 1731.4285338	test: 1746.7887988	best: 1746.7887988 (140)	total: 4m 39s	remaining: 28m 21s
160:	learn: 1715.6082148	test: 1733.2823605	best: 1733.2823605 (160)	total: 5m 18s	remaining: 27m 41s
180:	learn: 1701.0364108	test: 1720.6385509	best: 1720.6385509 (180)	total: 5m 58s	remaining: 27m


In [19]:
params2 = {'iterations': 500,
         'depth': 12,
         'loss_function': 'RMSE',
          'learning_rate': 0.1,
         'verbose': 20,
         'random_state': 12345}

In [20]:
cv(cv_cars, params2, fold_count=3, plot=False, as_pandas=True)

0:	learn: 6105.7160210	test: 6108.1252942	best: 6108.1252942 (0)	total: 6.11s	remaining: 50m 47s
20:	learn: 2015.3594744	test: 2044.8018882	best: 2044.8018882 (20)	total: 1m 44s	remaining: 39m 33s
40:	learn: 1684.4916469	test: 1737.6901164	best: 1737.6901164 (40)	total: 3m 23s	remaining: 37m 57s
60:	learn: 1611.2741092	test: 1683.3368226	best: 1683.3368226 (60)	total: 5m 8s	remaining: 36m 56s
80:	learn: 1569.0855266	test: 1658.0050906	best: 1658.0050906 (80)	total: 6m 57s	remaining: 36m 1s
100:	learn: 1533.7482596	test: 1639.6693209	best: 1639.6693209 (100)	total: 8m 53s	remaining: 35m 7s
120:	learn: 1505.7830709	test: 1626.5980686	best: 1626.5980686 (120)	total: 10m 45s	remaining: 33m 43s
140:	learn: 1480.2771930	test: 1614.9791863	best: 1614.9791863 (140)	total: 12m 40s	remaining: 32m 15s
160:	learn: 1460.9176818	test: 1606.1166782	best: 1606.1166782 (160)	total: 14m 34s	remaining: 30m 41s
180:	learn: 1445.1537394	test: 1599.8745407	best: 1599.8745407 (180)	total: 16m 27s	remaining: 

,iterations,test-RMSE-mean,test-RMSE-std,train-RMSE-mean,train-RMSE-std
0,0,6108.125294,34.534339,6105.716021,15.995754
1,1,5600.304138,39.436142,5597.223080,13.783993
2,2,5145.625796,37.013729,5140.883042,13.711362
3,3,4734.649569,39.740305,4729.499597,10.344687
4,4,4376.874875,33.400717,4370.610872,14.246881
...,...,...,...,...,...
495,495,1559.865423,13.415643,1282.736773,3.833117
496,496,1559.821778,13.377103,1282.622231,3.771516
497,497,1559.771385,13.345554,1282.263643,3.921894
498,498,1559.733276,13.367849,1282.013041,3.938931


Лучшая rmse получилась с depth=12.

Обучим модель из LightGBM тоже применив кросс-валидацию.

In [21]:
#Категориальные признаки преобразуем в формат "category"
features_train_lgb = features_train.copy()
features_train_lgb[cat_columns] = features_train_lgb[cat_columns].astype('category')

In [49]:
#Параметры модели
lgb_params = {'metric': 'rmse',
              'objective': 'regression',
             'num_boost_round': 2000,
             'stratified': False,
             'random_state': 12345,
             'max_depth': 12,
             'learning_rate': 0.1}

In [35]:
lgb_train = lgb.Dataset(features_train_lgb, target_train)

In [50]:
%%time
lgb.cv(lgb_params, lgb_train, verbose_eval=20)

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:503: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


[20]	cv_agg's rmse: 1963.68 + 7.35262
[40]	cv_agg's rmse: 1719.07 + 11.5202
[60]	cv_agg's rmse: 1666.06 + 12.0759
[80]	cv_agg's rmse: 1642.42 + 10.3916
[100]	cv_agg's rmse: 1628.74 + 10.8849
[120]	cv_agg's rmse: 1618.34 + 10.2438
[140]	cv_agg's rmse: 1610.1 + 10.2178
[160]	cv_agg's rmse: 1603.03 + 9.4201
[180]	cv_agg's rmse: 1597.4 + 8.848
[200]	cv_agg's rmse: 1593.09 + 9.53574
[220]	cv_agg's rmse: 1589.04 + 8.77717
[240]	cv_agg's rmse: 1585.56 + 8.78528
[260]	cv_agg's rmse: 1581.76 + 8.56311
[280]	cv_agg's rmse: 1578.83 + 8.66342
[300]	cv_agg's rmse: 1576.25 + 9.02427
[320]	cv_agg's rmse: 1573.52 + 9.12857
[340]	cv_agg's rmse: 1571.01 + 9.22936
[360]	cv_agg's rmse: 1568.73 + 9.16392
[380]	cv_agg's rmse: 1566.89 + 8.9799
[400]	cv_agg's rmse: 1565.07 + 9.08274
[420]	cv_agg's rmse: 1563.26 + 9.04258
[440]	cv_agg's rmse: 1561.63 + 9.02997
[460]	cv_agg's rmse: 1560.06 + 9.032
[480]	cv_agg's rmse: 1558.75 + 9.27422
[500]	cv_agg's rmse: 1557.62 + 9.38336
[520]	cv_agg's rmse: 1556.46 + 9.4085

{'rmse-mean': [4255.900841383357,
  3963.0126862777515,
  3704.1282649994428,
  3476.499012258806,
  3273.2290054921446,
  3096.020291520755,
  2936.238396247245,
  2793.4459905680437,
  2668.389071784849,
  2558.416597779897,
  2460.1071958099037,
  2374.939717672083,
  2298.836918989814,
  2231.2334871402445,
  2171.76776363191,
  2119.9259980530474,
  2074.105761976391,
  2032.8864578741413,
  1995.916150994767,
  1963.6818581747004,
  1934.5594064442491,
  1909.8699682633778,
  1886.9217001993609,
  1866.710591385768,
  1848.487522030243,
  1832.1861933264404,
  1818.1681466642572,
  1805.319724267835,
  1793.9594236721423,
  1783.5314298579003,
  1774.556161316787,
  1765.754818819581,
  1758.3819554421702,
  1751.2871239616156,
  1745.5034910166964,
  1739.2511363772269,
  1734.1368017266354,
  1728.7107706442396,
  1723.4936815779117,
  1719.067853445874,
  1714.6821634802914,
  1710.797380097464,
  1707.091448436097,
  1703.5030089154807,
  1700.5773339227278,
  1697.3050374017

In [51]:
lgb_params2 = {'metric': 'rmse',
              'objective': 'regression',
             'num_boost_round': 2000,
             'stratified': False,
             'random_state': 12345,
             'max_depth': 6}

In [52]:
%%time
lgb.cv(lgb_params2, lgb_train, verbose_eval=20)

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:503: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


[20]	cv_agg's rmse: 1969.37 + 8.81496
[40]	cv_agg's rmse: 1727.32 + 10.5987
[60]	cv_agg's rmse: 1677.04 + 10.5774
[80]	cv_agg's rmse: 1655.56 + 10.7546
[100]	cv_agg's rmse: 1642.13 + 9.62892
[120]	cv_agg's rmse: 1630.5 + 8.94109
[140]	cv_agg's rmse: 1621.53 + 8.06467
[160]	cv_agg's rmse: 1614.44 + 8.4425
[180]	cv_agg's rmse: 1608.01 + 8.70993
[200]	cv_agg's rmse: 1602.87 + 8.95125
[220]	cv_agg's rmse: 1598.6 + 9.18783
[240]	cv_agg's rmse: 1594.88 + 9.17773
[260]	cv_agg's rmse: 1591.4 + 9.7413
[280]	cv_agg's rmse: 1588.85 + 9.82778
[300]	cv_agg's rmse: 1586.56 + 9.78443
[320]	cv_agg's rmse: 1583.96 + 9.6814
[340]	cv_agg's rmse: 1581.82 + 9.32064
[360]	cv_agg's rmse: 1579.39 + 9.44444
[380]	cv_agg's rmse: 1576.85 + 9.51224
[400]	cv_agg's rmse: 1575.21 + 9.28255
[420]	cv_agg's rmse: 1572.84 + 9.12189
[440]	cv_agg's rmse: 1571 + 9.11862
[460]	cv_agg's rmse: 1569.46 + 8.76445
[480]	cv_agg's rmse: 1567.3 + 8.98747
[500]	cv_agg's rmse: 1565.77 + 9.17766
[520]	cv_agg's rmse: 1564.55 + 9.30724


{'rmse-mean': [4256.6327494343595,
  3964.4269640478306,
  3706.6037902967,
  3478.189876492179,
  3274.5444831739137,
  3098.359947889564,
  2938.1122840594294,
  2795.881909798233,
  2671.1733879138183,
  2560.834871924094,
  2464.4031291297397,
  2379.300690906097,
  2303.749771832713,
  2236.223125045991,
  2177.5226775883552,
  2125.600942676636,
  2079.1705984139553,
  2037.7408490531332,
  2001.2355297075205,
  1969.3679150094147,
  1941.1635888281012,
  1916.4467637015016,
  1894.0830520470531,
  1873.6575443527188,
  1856.2076302524506,
  1840.267480785916,
  1825.9315158244444,
  1813.0601895697437,
  1802.314886771722,
  1791.8379515998508,
  1782.4055747629313,
  1773.7967501892213,
  1766.8082604148908,
  1759.4488543324564,
  1752.9217410546348,
  1747.179649211781,
  1741.4534350776655,
  1736.4847390799441,
  1731.5662739411669,
  1727.319358019624,
  1723.228329795711,
  1719.7288896568975,
  1716.191463835558,
  1712.2110828406226,
  1709.2710846291097,
  1706.6485929

Лучшая модель обучилась при max_depth=12 и  num_boost_round=2000, делать большее количество итераций не имеет смысла, так как далее модель будет переобучаться.

Далее протестируем выбранные модели на тестовой выборке и проанализируем их скорость работы.

# 3. Анализ моделей

In [85]:
model_catboost = CatBoostRegressor(iterations=500,
         depth=12,
         loss_function='RMSE',
          learning_rate=0.1,
         verbose=20,
         random_state=12345)

In [86]:
%%time
model_catboost.fit(cv_cars)

0:	learn: 4250.3314892	total: 2.12s	remaining: 17m 38s
20:	learn: 1896.8860890	total: 42.9s	remaining: 16m 18s
40:	learn: 1672.6548981	total: 1m 26s	remaining: 16m 9s
60:	learn: 1615.5542228	total: 2m 10s	remaining: 15m 36s
80:	learn: 1574.4979440	total: 2m 56s	remaining: 15m 10s
100:	learn: 1545.5967848	total: 3m 44s	remaining: 14m 44s
120:	learn: 1520.2819612	total: 4m 31s	remaining: 14m 10s
140:	learn: 1495.4457260	total: 5m 17s	remaining: 13m 27s
160:	learn: 1476.9388231	total: 6m 4s	remaining: 12m 47s
180:	learn: 1464.4025942	total: 6m 51s	remaining: 12m 5s
200:	learn: 1448.9893512	total: 7m 35s	remaining: 11m 17s
220:	learn: 1435.9185533	total: 8m 25s	remaining: 10m 38s
240:	learn: 1422.6776386	total: 9m 15s	remaining: 9m 56s
260:	learn: 1410.9955395	total: 10m 2s	remaining: 9m 11s
280:	learn: 1399.5139431	total: 10m 48s	remaining: 8m 25s
300:	learn: 1389.1265998	total: 11m 38s	remaining: 7m 41s
320:	learn: 1380.2239315	total: 12m 27s	remaining: 6m 56s
340:	learn: 1371.3298503	to

In [87]:
%%time
predict = model_catboost.predict(pool_test)

CPU times: user 1.59 s, sys: 32.1 ms, total: 1.62 s
Wall time: 1.6 s


In [88]:
rmse_cat = mean_squared_error(target_test, predict)**.5
rmse_cat

1552.0264901942107

In [71]:
features_test_lgb = features_test.copy()
features_test_lgb[cat_columns] = features_test_lgb[cat_columns].astype('category')

In [80]:
lgb_params_test = {'metric': 'rmse',
              'objective': 'regression',
             #'num_boost_round': 2000,
             'stratified': False,
             'random_state': 12345,
             'max_depth': 12,
             'learning_rate': 0.1}

In [81]:
%%time
bst = lgb.train(lgb_params_test, lgb_train, num_boost_round=2000)

CPU times: user 2min 7s, sys: 402 ms, total: 2min 7s
Wall time: 2min 9s


In [82]:
%%time
predictions_lgb = bst.predict(features_test_lgb, num_iteration=bst.best_iteration)

CPU times: user 22.3 s, sys: 0 ns, total: 22.3 s
Wall time: 22.2 s


In [84]:
rmse_lgb = mean_squared_error(target_test, predictions_lgb)**.5
rmse_lgb

1523.455769415674

### Вывод

Catboost почти в 15 быстрее LightGBM делает предсказания (1.6 с и 22 с соответственно), однако время обучения модели Catboost значительно отстает: 20 мин против 2 мин у LGBM, при этом метрика rmse у LGBM ниже (лучше) чем у Catboost: rmse_lgb = 1523, rmse_cat = 1552. Возможно если увеличить количество итераций в обучении модели от Catboost ее качество будет лучше, но при этом и без того большое время обучения еще больше возрастет.